In [12]:
import tensorflow as tf
import keras
import numpy as np
import random
import pickle
import sys
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers import Dense
from keras.optimizers import Adam
from collections import deque
from keras.layers import Reshape

In [13]:
BOARD_ROWS = 7
BOARD_COLS = 7

In [14]:
# 참고 https://www.secmem.org/blog/2020/02/08/snake-dqn/

In [48]:
class Germ:
    def __init__(self, alpha = 0.02, gamma = 0.95, epsilon = 0.1):
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.board[0, 0] = self.board[BOARD_COLS-1, BOARD_COLS-1] = -1
        self.board[BOARD_ROWS-1, 0] = self.board[0, BOARD_COLS-1] = 1
        # 우리가 1, 선공일 때를 나타냄
        self.isEnd = False
        self.playerSymbol = 1
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.batch_size = 64
        self.min_replay_memory_size = 1000 # 얼마가 적당할지 잘 모르겠음.
        self.target_update_freq = 100

        self.model = self.build_model()
        self.target_model = self.build_model()
        self.target_model.set_weights(self.model.get_weights())
        self.model.summary()

        self.replay_memory_size = 5000
        self.replay_memory = deque(maxlen=self.replay_memory_size)
        self.target_update_counter = 0

    def cantmove(self): # 더 이상 움직일 수 없을 때 남은 곳을 상대 말로 채운다.
        self.isEnd = True
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i, j] == 0:
                    self.board[i, j] = -self.playerSymbol
        return None
  
    def winner(self): # 맵이 다 찼다면 점수를 반환한다.
        if sum(map(sum, map(abs, self.board))) == BOARD_ROWS*BOARD_COLS:
            self.isEnd = True
            return sum(map(sum, self.board))
        return None

    def availableActions(self): # 가능한 행동들을 반환한다.
        Actions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if self.board[i, j] == self.playerSymbol:
                    for ii in range(-2,3):
                        for jj in range(-2,3):
                            if ii == 0 and jj == 0:
                                continue
                            if i + ii < 0 or i + ii >= BOARD_ROWS or j + jj < 0 or j + jj >= BOARD_COLS:
                                continue
                            if self.board[i + ii, j + jj] == 0:
                                act = i
                                act = act*BOARD_COLS + j
                                act = act*BOARD_COLS + i + ii
                                act = act*BOARD_COLS + j + jj
                                Actions.append(act)
        return Actions

    def isAvailableAction(self, Action): # 가능한 행동인지?
        position = np.zeros(4)
        Action = int(Action)
        for i in range(4):
            position[3-i] = Action % BOARD_COLS
            Action = Action // BOARD_COLS
        position = [int(i) for i in position]
        return self.board[position[0]][position[1]]==self.playerSymbol and self.board[position[2]][position[3]]==0

    def updateState(self, Action): # 현재 상태에서 특정 행동을 한 다음 상태로 업데이트 한다.
        position = np.zeros(4)
        for i in range(4):
            position[3-i] = Action % BOARD_COLS
            Action = Action // BOARD_COLS
            position = [int(i) for i in position]
            #print(Action, position)
            ii = position[2] - position[0]
            jj = position[3] - position[1]
            if max(abs(ii), abs(jj)) == 2:
                self.board[position[0],position[1]] = 0

            dx1 = [-1, -1, -1, 0, 0, 1, 1, 1]
            dy1 = [-1, 0, 1, -1, 1, -1, 0, 1]
            i, j = position[2:4]
            self.board[i, j] = self.playerSymbol
            for ii, jj in zip(dx1, dy1):
                if i + ii < 0 or i + ii >= BOARD_ROWS or j + jj < 0 or j + jj >= BOARD_COLS:
                    continue
                if self.board[i + ii, j + jj] == -self.playerSymbol:
                    self.board[i + ii, j + jj] = self.playerSymbol

        # switch to another player
        self.playerSymbol = -self.playerSymbol

    def reset(self): # 리셋.
        self.board = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.board[0, 0] = self.board[BOARD_ROWS-1, BOARD_COLS-1] = 1
        self.board[BOARD_ROWS-1, 0] = self.board[0, BOARD_COLS-1] = -1
        self.boardHash = None
        self.isEnd = False
        self.playerSymbol = 1
    
        
    def showBoard(self):
    # p1: o  p2: x
        for i in range(0, BOARD_ROWS):
            print('------------------------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = 'o'
                if self.board[i, j] == -1:
                    token = 'x'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('------------------------------')  

    def build_model(self): # DQN 모델을 생성한다.
        model = Sequential()
        model.add(Conv2D(16, (3, 3), padding = 'valid', input_shape=(7, 7, 1), activation='relu'))
        model.add(Conv2D(16, (3, 3), padding = 'valid', input_shape=(5, 5, 1), activation='relu'))
        model.add(Conv2D(16, (3, 3), padding = 'valid', input_shape=(3, 3, 1), activation='relu'))
        model.add(Dense(64 * BOARD_COLS * BOARD_COLS, activation='relu'))
        model.add(Dense(BOARD_COLS**4, activation='relu'))
        model.add(Reshape((BOARD_ROWS**4,)))
        model.compile(loss='mse', optimizer=Adam(lr=self.alpha))
        return model

    def update_replay_memory(self, current_state, action, reward, next_state, done): # 리플레이 메모리에 상황을 저장한다.
        self.replay_memory.append((current_state, action, reward, next_state, done))

    def get_q_values(self, x): # 현재 상태에서 할 행동들의 q_value를 반환, x는 board에 대응됨.
        return self.model.predict(x.reshape(1,BOARD_ROWS, BOARD_COLS, 1))

    def getAction(self, state, epsilon):
        if np.random.rand() <= epsilon:
        # 무작위 행동 반환
            avac_size = len(self.availableActions())
            return self.availableActions()[random.randrange(avac_size)]
        else:
        # 모델로부터 행동 산출
            state = np.float32(state*self.playerSymbol)
            q_values = self.model.predict(state.reshape(1,BOARD_ROWS,BOARD_COLS,1))
            return np.argmax(q_values)

    def epsbyepi(self, episode):
        return max(self.epsilon, 1 - 1/(1+np.exp(-episode/15000+6)))

    def play(self, episode):
        prev_state = np.zeros((BOARD_ROWS,BOARD_COLS))
        self.reset()
        while not self.isEnd:
            self.showBoard()
            avac = self.availableActions()
            dora = False
            if not avac:
                self.cantmove()
            else:
                action = self.getAction(self.board, self.epsbyepi(episode))
                state = self.board * self.playerSymbol
                if self.isAvailableAction(action):
                    self.updateState(action)
                    win = self.winner()
                    if win is None:
                        reward = 0
                    else:
                        reward = win * self.playerSymbol
                    dora = True
                else:
                    self.isEnd = True
                    reward = -10
            if self.isEnd and dora:
                self.update_replay_memory(prev_state[0], prev_state[1], -reward, prev_state[2], False)
            prev_state = (state, action, self.board)
            self.update_replay_memory(state, action, reward, self.board, self.isEnd)
            print("")
            print("state:")
            print(state)
            print(seven.aTp(seven,action), reward)
            print("next:")
            print(self.board)


    def train(self):
        if len(self.replay_memory)<self.min_replay_memory_size: # 충분히 모이지 않으면 학습하지 않는다.
              return

        samples = random.sample(self.replay_memory, self.batch_size)
        current_input = np.stack([sample[0] for sample in samples]) # current_state들의 array
        current_q_values = self.model.predict(current_input.reshape(len(current_input),BOARD_ROWS, BOARD_COLS,1))
        next_input = np.stack([sample[3] for sample in samples])
        next_q_values = self.target_model.predict(next_input.reshape(len(next_input),BOARD_ROWS, BOARD_COLS,1))

        for i, (current_state, action, reward, _, done) in enumerate(samples):
            if done:
                next_q_value = reward
            else:
                next_q_value = reward + self.gamma * np.max(next_q_values[i])
            current_q_values[i, action] = next_q_value
        current_input = current_input.reshape((len(current_input),BOARD_ROWS,BOARD_COLS,1))
        hist = self.model.fit(current_input, current_q_values, batch_size=self.batch_size, verbose=1, shuffle=False)
        loss = hist.history['loss'][0]
        return loss

    def increase_target_update_counter(self): # target_model에 model을 업데이트한다. 그걸 세는 함수.
        self.target_update_counter += 1
        if self.target_update_counter >= self.target_update_freq:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0

    def save(self, model_filepath, target_model_filepath):
        self.model.save(model_filepath)
        self.target_model.save(target_model_filepath)

    def load(self, model_filepath, target_model_filepath):
        self.model = keras.models.load_model(model_filepath)
        self.target_model = keras.models.load_model(target_model_filepath)


In [49]:
dora = Germ()
dora.build_model()
# dora.load("./model/model_lr02_ep100000","./model/target_lr02_ep100000")
episode = 1


Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_81 (Conv2D)           (None, 5, 5, 16)          160       
_________________________________________________________________
conv2d_82 (Conv2D)           (None, 3, 3, 16)          2320      
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 1, 1, 16)          2320      
_________________________________________________________________
dense_54 (Dense)             (None, 1, 1, 3136)        53312     
_________________________________________________________________
dense_55 (Dense)             (None, 1, 1, 2401)        7531937   
_________________________________________________________________
reshape_27 (Reshape)         (None, 2401)              0         
Total params: 7,590,049
Trainable params: 7,590,049
Non-trainable params: 0
___________________________________________

In [50]:
dora.play(1)

------------------------------
| o |   |   |   |   |   | x | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
| x |   |   |   |   |   | o | 
------------------------------

state:
[[ 1.  0.  0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  1.]]
[6, 6, 4, 4] 0
next:
[[ 1.  0.  0.  0.  1.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.]]
------------------------------
| o |   |   |   | o |   | x | 
--------

| o | x | x | x |   |   |   | 
------------------------------
|   |   | x | x |   |   | x | 
------------------------------
| o | o |   | o | o |   |   | 
------------------------------

state:
[[ 0. -1. -1.  1.  1. -1. -1.]
 [ 0.  0.  0.  1.  1. -1. -1.]
 [ 0. -1.  0.  1.  1.  1.  0.]
 [-1.  0. -1. -1. -1.  0.  0.]
 [ 1. -1. -1. -1.  0.  0.  0.]
 [ 0.  0. -1. -1.  0.  0. -1.]
 [ 1.  1.  0.  1.  1.  0.  0.]]
[0, 4, 1, 2] 0
next:
[[ 0.  1.  1.  1.  0. -1. -1.]
 [ 0.  0.  1.  1.  1. -1. -1.]
 [ 0.  1.  0.  1.  1.  1.  0.]
 [-1.  0. -1. -1. -1.  0.  0.]
 [ 1. -1. -1. -1.  0.  0.  0.]
 [ 0.  0. -1. -1.  0.  0. -1.]
 [ 1.  1.  0.  1.  1.  0.  0.]]
------------------------------
|   | o | o | o |   | x | x | 
------------------------------
|   |   | o | o | o | x | x | 
------------------------------
|   | o |   | o | o | o |   | 
------------------------------
| x |   | x | x | x |   |   | 
------------------------------
| o | x | x | x |   |   |   | 
------------------------------
|   |   

[2, 5, 4, 6] 0
next:
[[ 1.  0.  1.  1.  1. -1. -1.]
 [ 1.  1.  1.  1.  1. -1. -1.]
 [ 1.  0.  1. -1. -1.  0. -1.]
 [ 1.  1.  1.  0. -1. -1. -1.]
 [ 1.  1. -1. -1.  0. -1. -1.]
 [ 1.  0. -1. -1.  0. -1.  0.]
 [ 1.  1. -1. -1. -1. -1.  1.]]
------------------------------
| o |   | o | o | o | x | x | 
------------------------------
| o | o | o | o | o | x | x | 
------------------------------
| o |   | o | x | x |   | x | 
------------------------------
| o | o | o |   | x | x | x | 
------------------------------
| o | o | x | x |   | x | x | 
------------------------------
| o |   | x | x |   | x |   | 
------------------------------
| o | o | x | x | x | x | o | 
------------------------------

state:
[[ 1.  0.  1.  1.  1. -1. -1.]
 [ 1.  1.  1.  1.  1. -1. -1.]
 [ 1.  0.  1. -1. -1.  0. -1.]
 [ 1.  1.  1.  0. -1. -1. -1.]
 [ 1.  1. -1. -1.  0. -1. -1.]
 [ 1.  0. -1. -1.  0. -1.  0.]
 [ 1.  1. -1. -1. -1. -1.  1.]]
[1, 1, 0, 1] 0
next:
[[ 1.  1.  1.  1.  1. -1. -1.]
 [ 1.  1.  1.  1. 

In [6]:
for i in range(100000):
  dora.play(episode)
  episode += 1
  dora.increase_target_update_counter()
  if i%1000==0 and i>0:
    print("round",i)
    dora.train()

round 1000
1/1 [==============================] - 0s 1ms/step - loss: 2.4761e-08
round 2000
1/1 [==============================] - 0s 1ms/step - loss: 2.1461e-08
round 3000
1/1 [==============================] - 0s 987us/step - loss: 2.2646e-08
round 4000
1/1 [==============================] - 0s 1ms/step - loss: 2.0445e-08
round 5000
1/1 [==============================] - 0s 1ms/step - loss: 2.2951e-08
round 6000
1/1 [==============================] - 0s 997us/step - loss: 6.5175e-04
round 7000
1/1 [==============================] - 0s 981us/step - loss: 4.3277e-08
round 8000
1/1 [==============================] - 0s 1ms/step - loss: 9.3966e-08
round 9000
1/1 [==============================] - 0s 1ms/step - loss: 1.4419e-07
round 10000
1/1 [==============================] - 0s 1ms/step - loss: 2.1791e-07
round 11000
1/1 [==============================] - 0s 1ms/step - loss: 2.5375e-07
round 12000
1/1 [==============================] - 0s 1ms/step - loss: 3.0170e-07
round 13000
1/1 [==

In [7]:
dora.save("./model/model_lr02_ep100000","./model/target_lr02_ep100000")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./model/model_lr02_ep100000/assets
INFO:tensorflow:Assets written to: ./model/target_lr02_ep100000/assets


In [8]:
dora = Germ()
dora.build_model()
dora.load("./model/model_lr02_ep100000","./model/target_lr02_ep100000")
episode = 100001
dora.save("./model/model_lr02_ep200000","./model/target_lr02_ep200000")

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 5, 5, 16)          160       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 3, 3, 16)          2320      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 1, 1, 16)          2320      
_________________________________________________________________
dense_6 (Dense)              (None, 1, 1, 3136)        53312     
_________________________________________________________________
dense_7 (Dense)              (None, 1, 1, 2401)        7531937   
_________________________________________________________________
reshape_3 (Reshape)          (None, 2401)              0         
Total params: 7,590,049
Trainable params: 7,590,049
Non-trainable params: 0
____________________________________________

In [ ]:
"""
from google.colab import files
files.download('model10000')
files.download('target10000')
"""

In [ ]:
# testing
!ls

In [5]:
class seven:
    def aTp(self,Action):
        position = np.zeros(4)
        Action = int(Action)
        for i in range(4):
            position[3-i] = Action % BOARD_COLS
            Action = Action // BOARD_COLS
            position = [int(i) for i in position]
        return position

In [6]:
class Player:
    def __init__(self,isHuman,playerSymbol):
        self.isHuman=isHuman
        self.model = self.build_model()
        self.playerSymbol = playerSymbol
    
    def build_model(self): # DQN 모델을 생성한다.
        model = Sequential()
        model.add(Conv2D(16, (3, 3), padding = 'valid', input_shape=(7, 7, 1), activation='relu'))
        model.add(Conv2D(16, (3, 3), padding = 'valid', input_shape=(5, 5, 1), activation='relu'))
        model.add(Conv2D(16, (3, 3), padding = 'valid', input_shape=(3, 3, 1), activation='relu'))
        model.add(Dense(64 * BOARD_COLS * BOARD_COLS, activation='relu'))
        model.add(Dense(BOARD_COLS**4, activation='relu'))
        model.add(Reshape((BOARD_ROWS**4,)))
        #model.compile(loss='mse', optimizer=Adam(lr=self.alpha))
        return model

    def availableActions(self,state): # 가능한 행동들을 반환한다.
        Actions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if state[i, j] == self.playerSymbol:
                    for ii in range(-2,3):
                        for jj in range(-2,3):
                            if ii == 0 and jj == 0:
                                continue
                            if i + ii < 0 or i + ii >= BOARD_ROWS or j + jj < 0 or j + jj >= BOARD_COLS:
                                continue
                            if state[i + ii, j + jj] == 0:
                                act = i
                                act = act*BOARD_COLS + j
                                act = act*BOARD_COLS + i + ii
                                act = act*BOARD_COLS + j + jj
                                Actions.append(act)
        return Actions    
    def isAvailableAction(self, state, Action): # 가능한 행동인지?
        position = np.zeros(4)
        Action = int(Action)
        for i in range(4):
            position[3-i] = Action % BOARD_COLS
            Action = Action // BOARD_COLS
            position = [int(i) for i in position]
        return state[position[0]][position[1]]==self.playerSymbol and state[position[2]][position[3]]==0

    
    def getAction(self,state):
        if self.isHuman:
            print("original row, col, target row, col")
            ro = int(input())
            co = int(input())
            rt = int(input())
            ct = int(input())
            if not ro>=0 and ro<7 and co>=0 and co<7 and rt>=0 and rt<7 and ct>=0 and ct<7:
                return None
            act = ro*7*7*7+co*7*7+rt*7+ct
            if not self.isAvailableAction(state,act):
                return None
            return act
        else:
            q_val = self.model.predict(state.reshape(1,BOARD_ROWS, BOARD_COLS, 1))
            avac = self.availableActions(state)
            if avac is None:
                return None
            avq = np.zeros(len(avac))
            for i,a in enumerate(avac):
                avq[i]=a
                p=seven.aTp(seven,a)
                print("(",p[0],",",p[1],") --> (",p[2],",",p[3],")   :  ",q_val[0,a])
            return avq[np.argmax(q_val[0,avac])]

    def load(self, model_filepath):
        self.model = keras.models.load_model(model_filepath)

In [7]:
ls

Charm.ipynb          Lunch_outside.ipynb  model/
Charm_Cracker.ipynb  README.md


In [8]:
class contest:
    def __init__(self,p1,p2):
        self.board = np.zeros((7,7))
        self.board[0,6]=self.board[6,0]=1
        self.board[0,0]=self.board[6,6]=-1
        self.p1=p1
        self.p2=p2
        self.isEnd = False
        
    
        
    def updateState(self, Action, p): # 현재 상태에서 특정 행동을 한 다음 상태로 업데이트 한다.
        dora = Action
        position = np.zeros(4)
        for i in range(4):
            position[3-i] = Action % BOARD_COLS
            Action = Action // BOARD_COLS
        position = [int(i) for i in position]
        ii = position[2] - position[0]
        jj = position[3] - position[1]
#         print("haha")
#         print(position)
#         print(ii,jj)
#         self.showBoard()
#         print(self.board[position[0:2]])
        if max(abs(ii), abs(jj)) == 2:
            self.board[position[0],position[1]] = 0
      
        dx1 = [-1, -1, -1, 0, 0, 1, 1, 1]
        dy1 = [-1, 0, 1, -1, 1, -1, 0, 1]
        i, j = position[2:4]
#         print("yay")
#         print(i,j)
#         self.showBoard()
        self.board[i, j] = p.playerSymbol
        for ii, jj in zip(dx1, dy1):
            if i + ii < 0 or i + ii >= BOARD_ROWS or j + jj < 0 or j + jj >= BOARD_COLS:
                continue
            if self.board[i + ii, j + jj] == -p.playerSymbol:
                self.board[i + ii, j + jj] = p.playerSymbol
        print("(",dora//(7**3)," ",(dora//(7**2))%7," ",(dora//7)%7," ",dora%7,")")
                
    def winner(self): # 맵이 다 찼다면 점수를 반환한다.
        if sum(map(sum, map(abs, self.board))) == BOARD_ROWS*BOARD_COLS:
            self.isEnd = True
            return sum(map(sum, self.board))
        return None

    def showBoard(self):
    # p1: o  p2: x
        for i in range(0, BOARD_ROWS):
            print('------------------------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = 'o'
                if self.board[i, j] == -1:
                    token = 'x'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('------------------------------')  
        
    def cantmove(self,p): # 더 이상 움직일 수 없을 때 남은 곳을 상대 말로 채운다.
        self.isEnd = True
        for i in range(BOARD_ROWS):
          for j in range(BOARD_COLS):
            if self.board[i, j] == 0:
              self.board[i, j] = -p.playerSymbol
        return None
        
    def start(self): # 게임을 시작한다.
        self.showBoard()
        while not self.isEnd:
            action1 = self.p1.getAction(self.board)
            if action1 is None:
                self.cantmove(self.p1)
                break
            self.updateState(action1,self.p1)
            self.showBoard()
            win=self.winner()
            if win is not None:
                self.isEnd = True
                print("player", int((3-np.sign(win))/2), "win")
                break
            action2 = self.p2.getAction(self.board)
            if action2 is None:
                self.cantmove(self.p2)
                break
            self.updateState(action2,self.p2)
            self.showBoard()
            win=self.winner()
            if win is not None:
                self.isEnd = True
                print("player", int((3-np.sign(win))/2), "win")
            
        
        

In [9]:
dora = Player(False,-1)
dora.load("./model/model_lr02_ep200000")
jiyoon = Player(True,1)
random = Player(False,1)
kapo = contest(random,dora)
kapo.start()

------------------------------
| x |   |   |   |   |   | o | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
| o |   |   |   |   |   | x | 
------------------------------
( 0 , 6 ) --> ( 0 , 4 )   :   0.0
( 0 , 6 ) --> ( 0 , 5 )   :   0.0
( 0 , 6 ) --> ( 1 , 4 )   :   0.0
( 0 , 6 ) --> ( 1 , 5 )   :   0.00023200951
( 0 , 6 ) --> ( 1 , 6 )   :   0.0
( 0 , 6 ) --> ( 2 , 4 )   :   0.0
( 0 , 6 ) --> ( 2 , 5 )   :   0.0
( 0 , 6 ) --> ( 2 , 6 )   :   0.002219474
( 6 , 0 ) --> ( 4 , 0 )   :   0.0
( 6 , 0 ) --> ( 4 , 1 )   :   0.0
( 6 , 0 ) --> ( 4 , 2 )   :   0.00033074198
( 6 , 0 ) --> ( 5 , 0 )   :   0.0
( 6 , 0 ) --> ( 5 , 1 )   :   0.0004511802
( 6 , 0 ) --> ( 5 , 2 )   :   0.0005229082
( 6 , 0 ) -->

( 2 , 6 ) --> ( 0 , 4 )   :   0.0
( 2 , 6 ) --> ( 0 , 5 )   :   0.0
( 2 , 6 ) --> ( 0 , 6 )   :   0.005442889
( 2 , 6 ) --> ( 1 , 4 )   :   0.0
( 2 , 6 ) --> ( 1 , 5 )   :   0.0
( 2 , 6 ) --> ( 1 , 6 )   :   0.0
( 2 , 6 ) --> ( 2 , 4 )   :   0.0032182918
( 2 , 6 ) --> ( 2 , 5 )   :   0.0
( 2 , 6 ) --> ( 3 , 4 )   :   0.0
( 2 , 6 ) --> ( 3 , 5 )   :   0.0
( 2 , 6 ) --> ( 3 , 6 )   :   0.0
( 2 , 6 ) --> ( 4 , 4 )   :   0.0
( 2 , 6 ) --> ( 4 , 5 )   :   0.0028976263
( 2 , 6 ) --> ( 4 , 6 )   :   0.0025383134
( 6 , 0 ) --> ( 4 , 0 )   :   0.0
( 6 , 0 ) --> ( 4 , 1 )   :   0.0
( 6 , 0 ) --> ( 4 , 2 )   :   0.0
( 6 , 0 ) --> ( 5 , 0 )   :   0.0
( 6 , 0 ) --> ( 5 , 1 )   :   0.0026497627
( 6 , 0 ) --> ( 5 , 2 )   :   0.0022674734
( 6 , 0 ) --> ( 6 , 1 )   :   0.00026110836
( 6 , 0 ) --> ( 6 , 2 )   :   0.006214213
( 6.0   0.0   6.0   2.0 )
------------------------------
| x | x |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------


( 0 , 0 ) --> ( 0 , 2 )   :   0.0
( 0 , 0 ) --> ( 1 , 0 )   :   0.0
( 0 , 0 ) --> ( 1 , 1 )   :   0.0
( 0 , 0 ) --> ( 1 , 2 )   :   0.0
( 0 , 0 ) --> ( 2 , 0 )   :   0.0
( 0 , 0 ) --> ( 2 , 1 )   :   0.0
( 0 , 0 ) --> ( 2 , 2 )   :   0.0
( 0 , 1 ) --> ( 0 , 2 )   :   0.15982427
( 0 , 1 ) --> ( 0 , 3 )   :   0.13828906
( 0 , 1 ) --> ( 1 , 0 )   :   0.005508151
( 0 , 1 ) --> ( 1 , 1 )   :   0.18262148
( 0 , 1 ) --> ( 1 , 2 )   :   0.11611005
( 0 , 1 ) --> ( 1 , 3 )   :   0.0
( 0 , 1 ) --> ( 2 , 0 )   :   0.03839626
( 0 , 1 ) --> ( 2 , 1 )   :   0.02161292
( 0 , 1 ) --> ( 2 , 2 )   :   0.18223792
( 0 , 1 ) --> ( 2 , 3 )   :   0.0
( 4 , 0 ) --> ( 2 , 0 )   :   0.22149536
( 4 , 0 ) --> ( 2 , 1 )   :   0.0
( 4 , 0 ) --> ( 2 , 2 )   :   0.023420248
( 4 , 0 ) --> ( 3 , 0 )   :   0.06815731
( 4 , 0 ) --> ( 3 , 1 )   :   0.21840161
( 4 , 0 ) --> ( 4 , 2 )   :   0.116863385
( 4 , 0 ) --> ( 5 , 0 )   :   0.013072836
( 4 , 0 ) --> ( 5 , 1 )   :   0.12742077
( 4 , 0 ) --> ( 5 , 2 )   :   0.04203274


( 4 , 3 ) --> ( 5 , 4 )   :   0.0
( 4 , 3 ) --> ( 5 , 5 )   :   0.0014813193
( 4 , 3 ) --> ( 6 , 1 )   :   0.0018285591
( 4 , 3 ) --> ( 6 , 2 )   :   0.0
( 4 , 3 ) --> ( 6 , 3 )   :   6.563318e-05
( 4 , 3 ) --> ( 6 , 4 )   :   0.0034141669
( 4 , 3 ) --> ( 6 , 5 )   :   0.0019879541
( 1.0   4.0   2.0   3.0 )
------------------------------
| x | x |   |   |   |   | o | 
------------------------------
|   |   |   |   | o |   |   | 
------------------------------
| x |   |   | o |   |   |   | 
------------------------------
|   | x |   | o |   |   |   | 
------------------------------
| x | x |   | o |   |   |   | 
------------------------------
|   |   |   |   |   |   |   | 
------------------------------
|   |   |   |   |   |   | x | 
------------------------------
( 0 , 0 ) --> ( 0 , 2 )   :   0.0
( 0 , 0 ) --> ( 1 , 0 )   :   0.0
( 0 , 0 ) --> ( 1 , 1 )   :   0.0
( 0 , 0 ) --> ( 1 , 2 )   :   0.0
( 0 , 0 ) --> ( 2 , 1 )   :   0.0
( 0 , 0 ) --> ( 2 , 2 )   :   0.0
( 0 , 1 ) --> ( 0 , 2 

( 0 , 0 ) --> ( 0 , 2 )   :   0.0
( 0 , 0 ) --> ( 1 , 0 )   :   0.0
( 0 , 0 ) --> ( 1 , 1 )   :   0.0
( 0 , 0 ) --> ( 1 , 2 )   :   0.0
( 0 , 0 ) --> ( 2 , 1 )   :   0.0
( 0 , 0 ) --> ( 2 , 2 )   :   0.0
( 0 , 1 ) --> ( 0 , 2 )   :   0.15982427
( 0 , 1 ) --> ( 0 , 3 )   :   0.13828906
( 0 , 1 ) --> ( 1 , 0 )   :   0.005508151
( 0 , 1 ) --> ( 1 , 1 )   :   0.18262148
( 0 , 1 ) --> ( 1 , 2 )   :   0.11611005
( 0 , 1 ) --> ( 1 , 3 )   :   0.0
( 0 , 1 ) --> ( 2 , 1 )   :   0.02161292
( 0 , 1 ) --> ( 2 , 2 )   :   0.18223792
( 2 , 0 ) --> ( 0 , 2 )   :   0.18312275
( 2 , 0 ) --> ( 1 , 0 )   :   0.0
( 2 , 0 ) --> ( 1 , 1 )   :   0.13548988
( 2 , 0 ) --> ( 1 , 2 )   :   0.0
( 2 , 0 ) --> ( 2 , 1 )   :   0.009226964
( 2 , 0 ) --> ( 2 , 2 )   :   0.087371334
( 2 , 0 ) --> ( 3 , 0 )   :   0.0
( 2 , 0 ) --> ( 4 , 0 )   :   0.19928658
( 2 , 0 ) --> ( 4 , 2 )   :   0.15539746
( 2 , 3 ) --> ( 0 , 2 )   :   0.14688697
( 2 , 3 ) --> ( 0 , 3 )   :   9.597554e-05
( 2 , 3 ) --> ( 0 , 4 )   :   0.0
( 2 , 

...

( 0 , 1 ) --> ( 1 , 3 )   :   0.0
( 0 , 2 ) --> ( 0 , 4 )   :   0.059115954
( 0 , 2 ) --> ( 1 , 3 )   :   0.013355727
( 0 , 3 ) --> ( 0 , 4 )   :   0.17636363
( 0 , 3 ) --> ( 1 , 3 )   :   0.035757102
( 0 , 3 ) --> ( 1 , 5 )   :   0.0
( 1 , 0 ) --> ( 3 , 0 )   :   0.0
( 1 , 1 ) --> ( 1 , 3 )   :   0.1634763
( 1 , 1 ) --> ( 3 , 0 )   :   0.0
( 1 , 2 ) --> ( 0 , 4 )   :   0.0
( 1 , 2 ) --> ( 1 , 3 )   :   0.0
( 1 , 2 ) --> ( 3 , 0 )   :   0.0
( 2 , 0 ) --> ( 3 , 0 )   :   0.0
( 2 , 0 ) --> ( 4 , 2 )   :   0.15539746
( 2 , 1 ) --> ( 1 , 3 )   :   0.16724315
( 2 , 1 ) --> ( 3 , 0 )   :   0.0
( 2 , 1 ) --> ( 4 , 2 )   :   0.09048368
( 2 , 2 ) --> ( 0 , 4 )   :   0.0023528554
( 2 , 2 ) --> ( 1 , 3 )   :   0.0
( 2 , 2 ) --> ( 3 , 0 )   :   0.023185672
( 2 , 2 ) --> ( 4 , 2 )   :   0.12435718
( 2 , 2 ) --> ( 4 , 4 )   :   0.1076211
( 3 , 1 ) --> ( 1 , 3 )   :   0.11220066
( 3 , 1 ) --> ( 3 , 0 )   :   0.10092833
( 3 , 1 ) --> ( 4 , 2 )   :   0.0
( 3 , 1 ) --> ( 5 , 0 )   :   0.013241667
( 3 , 

( 5 , 5 ) --> ( 3 , 5 )   :   0.0
( 5 , 5 ) --> ( 3 , 6 )   :   0.0
( 5 , 5 ) --> ( 4 , 4 )   :   0.0
( 5 , 5 ) --> ( 6 , 5 )   :   0.045885824
( 5 , 6 ) --> ( 3 , 5 )   :   0.0
( 5 , 6 ) --> ( 3 , 6 )   :   0.1401177
( 5 , 6 ) --> ( 4 , 4 )   :   0.0
( 5 , 6 ) --> ( 6 , 5 )   :   0.20064175
( 6 , 4 ) --> ( 4 , 4 )   :   0.094988145
( 6 , 4 ) --> ( 6 , 5 )   :   0.055646315
( 6 , 6 ) --> ( 4 , 4 )   :   0.0
( 6 , 6 ) --> ( 6 , 5 )   :   0.0
( 4.0   5.0   4.0   4.0 )
------------------------------
| x | x | x | x |   | o | o | 
------------------------------
| x | x | x |   | o | o |   | 
------------------------------
| x | x | x | o | o | o | o | 
------------------------------
|   | x | x | x | x |   |   | 
------------------------------
| x | x | x | x | x | x | x | 
------------------------------
|   | x | x | x | x | x | x | 
------------------------------
| o | o | o | o | x |   | x | 
------------------------------
( 0 , 5 ) --> ( 0 , 4 )   :   0.0014138243
( 0 , 5 ) --> ( 1 , 3

( 0 , 1 ) --> ( 1 , 3 )   :   0.0
( 0 , 2 ) --> ( 1 , 3 )   :   0.0133557385
( 0 , 5 ) --> ( 1 , 3 )   :   0.09862788
( 0 , 5 ) --> ( 1 , 6 )   :   0.1851765
( 0 , 5 ) --> ( 2 , 5 )   :   0.08077661
( 0 , 6 ) --> ( 1 , 6 )   :   0.0
( 0 , 6 ) --> ( 2 , 5 )   :   0.0
( 1 , 0 ) --> ( 3 , 0 )   :   0.0
( 1 , 1 ) --> ( 1 , 3 )   :   0.16347627
( 1 , 1 ) --> ( 3 , 0 )   :   0.0
( 1 , 2 ) --> ( 1 , 3 )   :   0.0
( 1 , 2 ) --> ( 3 , 0 )   :   0.0
( 1 , 5 ) --> ( 1 , 3 )   :   0.0
( 1 , 5 ) --> ( 1 , 6 )   :   0.22426131
( 1 , 5 ) --> ( 2 , 5 )   :   0.12728064
( 2 , 0 ) --> ( 3 , 0 )   :   0.0
( 2 , 1 ) --> ( 1 , 3 )   :   0.16724305
( 2 , 1 ) --> ( 3 , 0 )   :   0.0
( 2 , 2 ) --> ( 1 , 3 )   :   0.0
( 2 , 2 ) --> ( 3 , 0 )   :   0.023185601
( 2 , 4 ) --> ( 1 , 3 )   :   0.110304944
( 2 , 4 ) --> ( 1 , 6 )   :   0.120231725
( 2 , 4 ) --> ( 2 , 5 )   :   0.08205709
( 3 , 1 ) --> ( 1 , 3 )   :   0.11220053
( 3 , 1 ) --> ( 3 , 0 )   :   0.100928485
( 3 , 1 ) --> ( 5 , 0 )   :   0.0132415965
( 3 

( 0 , 1 ) --> ( 1 , 3 )   :   0.0
( 0 , 2 ) --> ( 1 , 3 )   :   0.013172108
( 0 , 5 ) --> ( 1 , 3 )   :   0.09919578
( 1 , 0 ) --> ( 3 , 0 )   :   0.0
( 1 , 1 ) --> ( 1 , 3 )   :   0.16294822
( 1 , 1 ) --> ( 3 , 0 )   :   0.0
( 1 , 2 ) --> ( 1 , 3 )   :   0.0
( 1 , 2 ) --> ( 3 , 0 )   :   0.0
( 1 , 4 ) --> ( 1 , 3 )   :   0.0
( 1 , 5 ) --> ( 1 , 3 )   :   0.0
( 2 , 0 ) --> ( 3 , 0 )   :   0.0
( 2 , 1 ) --> ( 1 , 3 )   :   0.16721234
( 2 , 1 ) --> ( 3 , 0 )   :   0.0
( 2 , 1 ) --> ( 4 , 3 )   :   0.1306312
( 2 , 2 ) --> ( 1 , 3 )   :   0.0
( 2 , 2 ) --> ( 3 , 0 )   :   0.02349345
( 2 , 2 ) --> ( 4 , 3 )   :   0.20032556
( 2 , 4 ) --> ( 1 , 3 )   :   0.1109192
( 2 , 4 ) --> ( 4 , 3 )   :   0.15274341
( 2 , 5 ) --> ( 1 , 3 )   :   0.16880992
( 2 , 5 ) --> ( 4 , 3 )   :   0.0036978535
( 3 , 1 ) --> ( 1 , 3 )   :   0.11194379
( 3 , 1 ) --> ( 3 , 0 )   :   0.10092472
( 3 , 1 ) --> ( 4 , 3 )   :   0.0
( 3 , 1 ) --> ( 5 , 0 )   :   0.013211943
( 3 , 2 ) --> ( 1 , 3 )   :   0.022244755
( 3 , 2 

( 1 , 2 ) --> ( 3 , 4 )   :   0.18127167
( 1 , 3 ) --> ( 3 , 4 )   :   0.0
( 1 , 4 ) --> ( 3 , 4 )   :   0.2321153
( 1 , 5 ) --> ( 3 , 4 )   :   0.09588862
( 1 , 6 ) --> ( 3 , 4 )   :   0.0
( 2 , 4 ) --> ( 3 , 4 )   :   0.028388936
( 2 , 5 ) --> ( 3 , 4 )   :   0.16834018
( 2 , 6 ) --> ( 3 , 4 )   :   0.023368916
( 4 , 4 ) --> ( 3 , 4 )   :   0.18166256
( 5 , 2 ) --> ( 3 , 4 )   :   0.0
( 5 , 3 ) --> ( 3 , 4 )   :   0.0824351
( 5 , 4 ) --> ( 3 , 4 )   :   0.19379416
( 1.0   4.0   3.0   4.0 )
------------------------------
| x | x | x | x | x | x | x | 
------------------------------
| x | x | x | x |   | x | x | 
------------------------------
| x | o | o | x | x | x | x | 
------------------------------
| x | o | o | x | x | x | o | 
------------------------------
| x | o | o | x | x | x | o | 
------------------------------
| o | o | x | x | x | o | o | 
------------------------------
| o | o | x | x | x | o | o | 
------------------------------
( 2 , 2 ) --> ( 1 , 4 )   :   0.017702

###### 

[[ 0. -1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]
[[-0.  1. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -1. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0.]]
